In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [31]:
class nfl_dataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]
        
        if self.transform:
            sample = self.transform(sample)
        
        sample = torch.tensor(sample, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)
        
        return sample, target

In [17]:
pd.set_option('display.max_columns', None)
data = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\nfl_data\data_mdn.csv', delimiter=',')
data.head(1)

,Key,Player,Year,Age,Exp_3,Exp_2,Exp_1,num_games_3,num_games_2,num_games_1,G_3,G_2,G_1,GS_3,GS_2,GS_1,Pass_Cmp_3,Pass_Cmp_2,Pass_Cmp_1,Pass_Att_3,Pass_Att_2,Pass_Att_1,Pass_Yds_3,Pass_Yds_2,Pass_Yds_1,Pass_TD_3,Pass_TD_2,Pass_TD_1,Pass_Int_3,Pass_Int_2,Pass_Int_1,Rush_Att_3,Rush_Att_2,Rush_Att_1,Rush_Yds_3,Rush_Yds_2,Rush_Yds_1,Rush_TD_3,Rush_TD_2,Rush_TD_1,Rec_Tgt_3,Rec_Tgt_2,Rec_Tgt_1,Rec_Rec_3,Rec_Rec_2,Rec_Rec_1,Rec_Yds_3,Rec_Yds_2,Rec_Yds_1,Rec_TD_3,Rec_TD_2,Rec_TD_1,Fmb_3,Fmb_2,Fmb_1,FmbLost_3,FmbLost_2,FmbLost_1,ProBowl_3,ProBowl_2,ProBowl_1,AllPro_3,AllPro_2,AllPro_1,Pos_RB,Pos_TE,Pos_WR,Points_half-ppr
0,AbbrJa00,Jared Abbrederis,2016,26,-1,-1,0,0,0,16,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,16.0,0,0,9,0,0,111,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,1.3


In [29]:
features = data.iloc[:, 2:-1]
targets = data.iloc[:, -1:]
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
train_dataset = nfl_dataset(X_train_scaled, y_train_scaled)
test_dataset = nfl_dataset(X_test_scaled, y_test_scaled)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [38]:
X_train_scaled.shape

(17872, 65)

In [ ]:
class MDN(nn.Module):
    def __init__(self, input_size, num_gaussians):
        super(MDN, self).__init__()
        self.num_gaussians = num_gaussians
        
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        
        self.relu = nn.ReLU()
        
        self.pi_layer = nn.Linear(64, num_gaussians)
        self.mu_layer = nn.Linear(64, num_gaussians)
        self.sigma_layer = nn.Linear(64, num_gaussians)
        
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        pi = self.softmax(self.pi_layer(x))
        mu = self.mu_layer(x)
        sigma = torch.exp(self.sigma_layer(x))
    
    def mdn_loss(pi, mu, sigma, target):
        